# 0.0. Imports

In [49]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

## 0.1. Loading Data

In [2]:
df_store_raw = pd.read_csv('../data/store.csv')
df_sales_raw = pd.read_csv('../data/train.csv', low_memory=False)

# merge
df_raw = pd.merge(df_store_raw, df_sales_raw, how='left', on='Store')

## 0.2. Helper Functions

In [3]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    sns.set_style("whitegrid")
    
    sns.set()

In [4]:
jupyter_settings();

Populating the interactive namespace from numpy and matplotlib


## 1.0. Data Description

In [5]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [6]:
cols_new = ['store', 'store_type', 'assortment', 'competition_distance',
            'competition_open_since_month', 'competition_open_since_year', 'promo2',
            'promo2_since_week', 'promo2_since_year', 'promo_interval',
            'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
            'state_holiday', 'school_holiday']

df1.columns = cols_new

## 1.2. Data Dimensions

In [7]:
print(f'Number of Rows: {df1.shape[0]}')
print(f'Number of Columns: {df1.shape[1]}')

Number of Rows: 1017209
Number of Columns: 18


## 1.3. Data Types

In [8]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
dtype: object

## 1.4. Check NA

In [9]:
df1.isna().sum().sort_values(ascending=False)

promo_interval                  508031
promo2_since_week               508031
promo2_since_year               508031
competition_open_since_month    323348
competition_open_since_year     323348
competition_distance              2642
sales                                0
state_holiday                        0
promo                                0
open                                 0
customers                            0
store                                0
date                                 0
day_of_week                          0
store_type                           0
promo2                               0
assortment                           0
school_holiday                       0
dtype: int64

## 1.5. Fillout NA

In [10]:
# promo_interval 
df1['promo_interval'].fillna(0, inplace=True)

month_map = {1: 'Jan', 2: 'Feb',3: 'Mar',4: 'Apr',5: 'May',6: 'Jun',7: 'Jul', 8: 'Aug',9: 'Sept',10: 'Oct',11: 'Nov', 12: 'Dec'}
df1['month_promo'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1.apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_promo'] in x['promo_interval'].split(',') else 0, axis=1)

# promo2_since_week,  promo2_since_year
df1['promo2_since_year'] = df1[['promo2_since_year', 'date']].apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)
df1['promo2_since_week'] = df1[['promo2_since_week', 'date']].apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# competition_open_since_year, competition_open_since_month
df1['competition_open_since_year'] = df1[['competition_open_since_year', 'date']].apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)
df1['competition_open_since_month'] = df1[['competition_open_since_month', 'date']].apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_distance
df1['competition_distance'].fillna(200000.0, inplace=True)

## 1.6. Change Data Types

In [11]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.7. Descriptive Statistics

In [12]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(include=['object'])

### 1.7.1. Numerical Attributes

In [13]:
# Central Tendency: mean, median
c1 = pd.DataFrame(num_attributes.apply(np.mean))
c2 = pd.DataFrame(num_attributes.apply(np.median))

# Dispersion: min, max, range, std, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(min))
d2 = pd.DataFrame(num_attributes.apply(max))
d3 = pd.DataFrame(num_attributes.apply(lambda x: np.max(x) - np.min(x)))
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.std()))
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew()))
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis()))

# concat
m = pd.concat([d1,d2,d3,c1,c2,d4,d5,d6], axis=1).reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,store,1.00,1115.00,1114.00,558.43,558.00,321.91,-0.00,-1.20
1,competition_distance,20.00,200000.00,199980.00,5935.44,2330.00,12547.65,10.24,147.79
2,competition_open_since_month,1.00,12.00,11.00,6.79,7.00,3.31,-0.04,-1.23
3,competition_open_since_year,1900.00,2015.00,115.00,2010.32,2012.00,5.52,-7.24,124.07
4,promo2,0.00,1.00,1.00,0.50,1.00,0.50,-0.00,-2.00
5,promo2_since_week,1.00,52.00,51.00,23.62,22.00,14.31,0.18,-1.18
6,promo2_since_year,2009.00,2015.00,6.00,2012.79,2013.00,1.66,-0.78,-0.21
7,day_of_week,1.00,7.00,6.00,4.00,4.00,2.00,0.00,-1.25
8,sales,0.00,41551.00,41551.00,5773.82,5744.00,3849.93,0.64,1.78
9,customers,0.00,7388.00,7388.00,633.15,609.00,464.41,1.60,7.09


### 1.7.2. Categorical Attributes

In [14]:
cat_attributes.describe(include=['object'])

,store_type,assortment,promo_interval,state_holiday,month_promo
count,1017209,1017209,1017209,1017209,1017209
unique,4,3,4,4,12
top,a,a,0,0,Mar
freq,551627,537445,508031,986159,103695


# 2.0. Feature Engineering

In [47]:
df2 = df1.copy()

## 2.* Feature Engineering

In [48]:
# date: year, month, day, week_of_year, year_week
df2['year'] = df2['date'].dt.year
df2['month'] = df2['date'].dt.month
df2['day'] = df2['date'].dt.day
df2['week_of_year'] = df2['date'].dt.isocalendar().week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

# competition_open_since_year  - competition_open_since_month
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

# promo2_since_year, promo2_since_week
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w'))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

# public_holiday
state_holiday_dict = {'a' : 'public_holiday', 'b' : 'easter_holiday', 'c' : 'christmas', '0' : 'regular_day'}
df2['state_holiday'] = df2['state_holiday'].map(state_holiday_dict)

# assortment
assortment_dict = {'a' : 'basic', 'b' : 'extra', 'c' : 'extended'}
df2['assortment'] = df2['assortment'].map(assortment_dict)

# 3.0. Data Filtering